In [1]:
import scanpy as sc
import pandas as pd
import scanpy.external as sce
import anndata as ad


In [2]:
import sccoda
import importlib
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt

from sccoda.util import comp_ana as mod
from sccoda.util import cell_composition_data as dat
from sccoda.util import data_visualization as viz

import sccoda.datasets as scd

2025-05-25 22:55:34.830178: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-25 22:55:34.831615: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-25 22:55:34.836028: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-25 22:55:34.848918: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748231734.864808 1661034 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748231734.86

In [160]:
cell_counts_CRC = pd.read_excel('cellcounts.xlsx', sheet_name='CRC')

In [161]:
cell_counts_CRC 

,Sample,C1,C2,C3,C4,C5
0,CTL,103,424,321,27,7
1,anti-PD1,239,432,377,27,1


In [162]:
data_CRC = dat.from_pandas(cell_counts_CRC, covariate_columns=["Sample"])
data_CRC

AnnData object with n_obs × n_vars = 2 × 5
    obs: 'Sample'

In [168]:
model_CRC = mod.CompositionalAnalysis(data_CRC, formula="Sample", reference_cell_type='C5')

In [169]:
sim_results_CRC = model_CRC.sample_hmc()

100%|██████████| 20000/20000 [00:39<00:00, 502.88it/s]


MCMC sampling finished. (54.592 sec)
Acceptance rate: 50.4%


In [170]:
sim_results_CRC.set_fdr(est_fdr=0.4)
sim_results_CRC.summary()

Compositional Analysis summary:

Data: 2 samples, 5 cell types
Reference index: 4
Formula: Sample

Intercepts:
           Final Parameter  Expected Sample
Cell Type                                  
C1                   3.064       133.666147
C2                   4.288       454.566970
C3                   4.047       357.217647
C4                   1.521        28.569658
C5                  -0.226         4.979579


Effects:
                              Final Parameter  Expected Sample  \
Covariate          Cell Type                                     
Sample[T.anti-PD1] C1                 0.52871       207.096225   
                   C2                 0.00000       415.080928   
                   C3                 0.00000       326.187871   
                   C4                 0.00000        26.087950   
                   C5                 0.00000         4.547027   

                              log2-fold change  
Covariate          Cell Type                    
Sample[T.

In [171]:
print(sim_results_CRC.credible_effects())

Covariate           Cell Type
Sample[T.anti-PD1]  C1            True
                    C2           False
                    C3           False
                    C4           False
                    C5           False
Name: Final Parameter, dtype: bool


## FMC

In [134]:
cell_counts_FMC = pd.read_excel('cellcounts.xlsx', sheet_name='FMC')
cell_counts_FMC

,Sample,C1,C2,C3,C5,C6
0,CTL,78,54,342,34,3
1,anti-PD1,226,176,570,34,39
2,anti-Tigit,19,16,102,11,1


In [135]:
data_FMC = dat.from_pandas(cell_counts_FMC, covariate_columns=["Sample"])
data_FMC

AnnData object with n_obs × n_vars = 3 × 5
    obs: 'Sample'

In [136]:
model_FMC = mod.CompositionalAnalysis(data_FMC, formula="Sample", reference_cell_type="automatic")

Automatic reference selection! Reference cell type set to C5


In [137]:
sim_results_FMC = model_FMC.sample_hmc()

100%|██████████| 20000/20000 [00:41<00:00, 477.32it/s]


MCMC sampling finished. (56.270 sec)
Acceptance rate: 52.7%


In [138]:
sim_results_FMC.set_fdr(est_fdr=0.4)
sim_results_FMC.summary()

Compositional Analysis summary:

Data: 3 samples, 5 cell types
Reference index: 3
Formula: Sample

Intercepts:
           Final Parameter  Expected Sample
Cell Type                                  
C1                   2.673        91.174600
C2                   2.406        69.809936
C3                   4.069       368.255265
C5                   1.589        30.838884
C6                   0.271         8.254647


Effects:
                                Final Parameter  Expected Sample  \
Covariate            Cell Type                                     
Sample[T.anti-PD1]   C1                     0.0        91.174600   
                     C2                     0.0        69.809936   
                     C3                     0.0       368.255265   
                     C5                     0.0        30.838884   
                     C6                     0.0         8.254647   
Sample[T.anti-Tigit] C1                     0.0        91.174600   
                     C2   

In [139]:
print(sim_results_FMC.credible_effects())

Covariate             Cell Type
Sample[T.anti-PD1]    C1           False
                      C2           False
                      C3           False
                      C5           False
                      C6           False
Sample[T.anti-Tigit]  C1           False
                      C2           False
                      C3           False
                      C5           False
                      C6           False
Name: Final Parameter, dtype: bool


## LCMV treatment

In [141]:
cell_counts_LCMV = pd.read_excel('cellcounts.xlsx', sheet_name='LCMV')
cell_counts_LCMV

,Sample,C1,C2,C3,C4,C5,C6
0,CTL,700,1269,550,333,212,204
1,anti-PD1,273,250,36,101,23,7


In [143]:
data_LCMV = dat.from_pandas(cell_counts_LCMV, covariate_columns=["Sample"])
data_LCMV

AnnData object with n_obs × n_vars = 2 × 6
    obs: 'Sample'

In [172]:
model_LCMV = mod.CompositionalAnalysis(data_LCMV, formula="Sample", reference_cell_type="automatic")

Automatic reference selection! Reference cell type set to C2


In [146]:
sim_results_LCMV = model_LCMV.sample_hmc()

100%|██████████| 20000/20000 [00:38<00:00, 513.77it/s]


MCMC sampling finished. (51.330 sec)
Acceptance rate: 53.6%


In [154]:
sim_results_LCMV.set_fdr(est_fdr=0.4)
sim_results_LCMV.summary()

Compositional Analysis summary:

Data: 2 samples, 6 cell types
Reference index: 1
Formula: Sample

Intercepts:
           Final Parameter  Expected Sample
Cell Type                                  
C1                   2.221       529.289175
C2                   2.618       787.241393
C3                   1.410       235.223218
C4                   1.437       241.660761
C5                   0.647       109.676478
C6                   0.279        75.908975


Effects:
                              Final Parameter  Expected Sample  \
Covariate          Cell Type                                     
Sample[T.anti-PD1] C1                0.466403       728.099388   
                   C2                0.000000       679.280808   
                   C3                0.000000       202.965214   
                   C4                0.000000       208.519926   
                   C5                0.000000        94.635683   
                   C6                0.000000        65.498982  

In [155]:
print(sim_results_LCMV.credible_effects())

Covariate           Cell Type
Sample[T.anti-PD1]  C1            True
                    C2           False
                    C3           False
                    C4           False
                    C5           False
                    C6           False
Name: Final Parameter, dtype: bool


## LCMV acute chronic

In [150]:
cell_counts = pd.read_excel('cellcounts.xlsx', sheet_name='LCMV_ac')
cell_counts = cell_counts.iloc[:2].copy()
cell_counts[cell_counts.select_dtypes(include='number').columns] = cell_counts.select_dtypes(include='number').astype(int)
cell_counts

,Sample,C1,C2,C3,C4,C5,C6
0,Acute,109,725,88,7,70,166
1,Chronic,113,501,232,35,25,197


In [151]:
data = dat.from_pandas(cell_counts, covariate_columns=["Sample"])
data

AnnData object with n_obs × n_vars = 2 × 6
    obs: 'Sample'

In [173]:
model_LCMV_ac = mod.CompositionalAnalysis(data, formula="Sample", reference_cell_type="automatic")

Automatic reference selection! Reference cell type set to C1


In [153]:
sim_results_LCMV_ac = model_LCMV_ac.sample_hmc()

100%|██████████| 20000/20000 [00:39<00:00, 502.68it/s]


MCMC sampling finished. (51.670 sec)
Acceptance rate: 49.3%


In [174]:
sim_results_LCMV_ac.set_fdr(est_fdr=0.4)
sim_results_LCMV_ac.summary()

Compositional Analysis summary:

Data: 2 samples, 6 cell types
Reference index: 0
Formula: Sample

Intercepts:
           Final Parameter  Expected Sample
Cell Type                                  
C1                   1.767       112.096698
C2                   3.542       661.402030
C3                   1.815       117.608566
C4                  -0.036        18.473954
C5                   0.960        50.016946
C6                   2.209       174.401806


Effects:
                             Final Parameter  Expected Sample  \
Covariate         Cell Type                                     
Sample[T.Chronic] C1                0.000000       121.781018   
                  C2               -0.340057       511.408015   
                  C3                0.574927       227.045745   
                  C4                0.400584        29.958359   
                  C5                0.000000        54.338038   
                  C6                0.000000       189.468824   

     

In [175]:
print(sim_results_LCMV_ac.credible_effects())

Covariate          Cell Type
Sample[T.Chronic]  C1           False
                   C2            True
                   C3            True
                   C4            True
                   C5           False
                   C6           False
Name: Final Parameter, dtype: bool
